# SETUP CELLS

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import requests
import json

# === Setup Paths ===
sys.path.insert(0, os.path.abspath("./genie-worksheets/src"))
root_dir = os.getcwd()

# === Load Environment ===
env_file = os.path.join(root_dir, ".env")
load_dotenv(env_file)

# === Verify API Key ===
AGENT_KEY = os.getenv("LLM_API_KEY")
BASE_URL = os.getenv("LLM_API_BASE_URL")
DATABASE_KEY = os.getenv("EXA_API_KEY")

# print(AGENT_KEY)


if not AGENT_KEY or AGENT_KEY == "your_api_key_here":
    raise ValueError("❌ Please set your LLM_API_KEY in .env file")

print("✓ Environment loaded")
print(f"✓ API Base URL: {BASE_URL}")


# === Import only what you need from Genie ===
from worksheets.specification.from_spreadsheet import gsheet_to_classes
print("✓ Genie Worksheets imported")

✓ Environment loaded
✓ API Base URL: https://api.openai.com/v1
✓ Genie Worksheets imported


In [2]:
# Ensure Environment is actually working
import requests
import json
from dotenv import load_dotenv

response = requests.post(
    f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {AGENT_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "user", "content": "Say hi in one word"}
        ],
        "max_tokens": 5
    }
)

if response.status_code == 200:
    result = response.json()
    print("Success!")
    print(f"Response: {result['choices'][0]['message']['content']}")
    print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
else:
    print(f"Error {response.status_code}: {response.text}")

Success!
Response: Hello!
Tokens used: 14


In [3]:
current_dir = os.getcwd()
root_dir = os.path.join(current_dir, "genie-worksheets")

# Credentials for reading the spreadsheet
!curl -L -o creds.zip "https://drive.google.com/uc?export=download&id=11QvSs2JZ5qpPrCvbX66Dg8pxfM_B8GaH"
!unzip creds.zip -d genie-worksheets/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3009  100  3009    0     0   1863      0  0:00:01  0:00:01 --:--:-- 10595
Archive:  creds.zip
  inflating: genie-worksheets/worksheets/token.json  
  inflating: genie-worksheets/worksheets/credentials.json  
  inflating: genie-worksheets/worksheets/service_account.json  


In [4]:
!mv "genie-worksheets/worksheets/credentials.json" "genie-worksheets/src/worksheets/config/credentials.json"
!mv "genie-worksheets/worksheets/service_account.json" "genie-worksheets/src/worksheets/config/service_account.json"
!mv "genie-worksheets/worksheets/token.json" "genie-worksheets/src/worksheets/config/token.json"

creds1 = os.path.join(root_dir, "src", "worksheets", "config", "credentials.json")
creds2 = os.path.join(root_dir, "src", "worksheets", "config", "service_account.json")
creds3 = os.path.join(root_dir, "src", "worksheets", "config", "token.json")

for cred in [creds1, creds2, creds3]:
  assert os.path.exists(cred), f"Cannot find the credential file: {cred}"

In [ ]:
from worksheets.agent.config import agent_api
import json

# Load the full Celeste Chapter 1 database
with open('final_chapter1_info.json', 'r') as f:
    chapter1_data = json.load(f)

CELESTE_COLLECTIBLES = chapter1_data['collectibles']

# Helper function to get next strawberry ID from count
def get_next_strawberry_id(num_collected: int) -> str:
    """
    Convert number of strawberries collected to the ID of the next strawberry.
    
    Args:
        num_collected: Number of strawberries already collected (0-19)
        
    Returns:
        The collectible_id of the next strawberry (e.g., "strawberry_1", "strawberry_11")
    """
    # Filter only strawberries (exclude crystal_heart and cassette_tape)
    strawberries = [
        c for c in CELESTE_COLLECTIBLES 
        if c['collectible_type'] == 'strawberry'
    ]
    
    # Sort by collectible_number to ensure correct order
    strawberries.sort(key=lambda x: x['collectible_number'])
    
    # Get the next strawberry (num_collected + 1)
    next_index = num_collected  # If 10 collected, index 10 is the 11th strawberry
    
    if next_index < 0 or next_index >= len(strawberries):
        return None
    
    return strawberries[next_index]['collectible_id']

# Register API function for getting hints
@agent_api("get_hint", "Get a hint for a Celeste collectible")
def get_hint(
    collectible_id: str, 
    hint_tier: int
):
    """
    Get a hint for a specific collectible.
    
    Args:
        collectible_id: The collectible ID (e.g., 'strawberry_11', 'crystal_heart')
                       OR a number (0-19) representing strawberries collected
        hint_tier: Which hint tier to return (1, 2, or 3)
    """
    # Check if collectible_id is actually a number (strawberry count)
    # This handles cases like collectible_id = "10"
    try:
        num_as_int = int(collectible_id)
        # If it's a valid number, convert to strawberry ID
        if 0 <= num_as_int <= 19:
            actual_id = get_next_strawberry_id(num_as_int)
            if actual_id:
                collectible_id = actual_id
                print(f"DEBUG: Converted {num_as_int} → {collectible_id}")
    except (ValueError, TypeError):
        # Not a number, use as-is
        pass
    
    # Find the collectible
    collectible = next(
        (item for item in CELESTE_COLLECTIBLES
         if item["collectible_id"] == collectible_id),
        None
    )
    
    if not collectible:
        return {
            "status": "error",
            "message": f"Collectible '{collectible_id}' not found."
        }
    
    # Validate hint_tier (1-3 maps to index 0-2)
    num_sources = len(collectible.get("info", []))
    if hint_tier < 1 or hint_tier > num_sources:
        return {
            "status": "error",
            "message": f"Hint tier {hint_tier} not available for {collectible_id}. Available tiers: 1-{num_sources}"
        }
    
    # Get the hint for this tier (tier 1 = index 0, tier 2 = index 1, etc.)
    hint_info = collectible["info"][hint_tier - 1]
    
    return {
        "status": "success",
        "collectible_id": collectible_id,
        "collectible_type": collectible.get("collectible_type", "unknown"),
        "collectible_number": collectible.get("collectible_number"),
        "hidden": collectible.get("hidden", False),
        "section": collectible.get("section", "unknown"),
        "hint_tier": hint_tier,
        "hint": hint_info["instruction"],
        "source": hint_info["source"]
    }

# Test the conversion
print("Testing get_next_strawberry_id():")
print(f"  0 strawberries collected → {get_next_strawberry_id(0)}")
print(f"  10 strawberries collected → {get_next_strawberry_id(10)}")
print(f"  19 strawberries collected → {get_next_strawberry_id(19)}")
print(f"  20 strawberries collected → {get_next_strawberry_id(20)}")

print("\nTesting get_hint with number input:")
result = get_hint("10", 1)
print(f"  get_hint('10', 1) → {result['collectible_id']}")

In [17]:
# Enter the ID of your google sheet here
gsheet_id_default = "1kpbfZkxR288BmRQ7oxaIbGfPrP4xgqDrAcx8OQz6qcg"

Ensure to share with the following account: ``yelpbotvm@mixedinitiative.iam.gserviceaccount.com``

In [ ]:
botname = "CelesteCollectibleGenie"
starting_prompt = "Hello! I am CelesteCollectibleGenie. I can help you find collectibles in Celeste Chapter 1. Tell me the collectible ID (like 'strawberry_11') or a number (0-19 for how many strawberries you've collected), and I'll give you hints!"
description = "CelesteCollectibleGenie helps players collect strawberries in Celeste Chapter 1. Users can request hints by providing either: (1) a collectible ID like 'strawberry_11', or (2) a number 0-19 representing how many strawberries they've already collected. When given a number, the assistant automatically determines which strawberry the player needs next. The assistant provides tiered hints (1-3) with increasing detail to help players without spoiling the solution completely."

In [19]:
# Helper code to convert the dialogue state to JSON format

from worksheets.utils.annotation import get_agent_action_schemas, get_context_schema
from worksheets.core.dialogue import CurrentDialogueTurn

def convert_to_json(dialogue: list[CurrentDialogueTurn]):
    json_dialogue = []
    for turn in dialogue:
        json_turn = {
            "user": turn.user_utterance,
            "bot": turn.system_response,
            "turn_context": get_context_schema(turn.context),
            "global_context": get_context_schema(turn.global_context),
            "system_action": get_agent_action_schemas(turn.system_action),
            "user_target_sp": turn.user_target_sp,
            "user_target": turn.user_target,
            "user_target_suql": turn.user_target_suql,
        }
        json_dialogue.append(json_turn)
    return json_dialogue

In [20]:
# You need to define the models to use for each componenet of Genie Worksheets.
# You can also load the configurations from a yaml file.

from worksheets.agent.config import Config, AzureModelConfig, OpenAIModelConfig

config = Config(
    semantic_parser = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    response_generator = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    knowledge_parser=OpenAIModelConfig(),
    knowledge_base=OpenAIModelConfig(),
    validate_response= False,

    prompt_log_path = "logs.log",
    conversation_log_path= "conv_log.json",
)

In [21]:
# Initialize your llm by providing it the prompts directory and your `.env` file which contains the secrets

from worksheets.llm.prompts import init_llm

# Use paths relative to current directory
prompts_path = os.path.join(os.getcwd(), "genie-worksheets", "src", "worksheets", "prompts")
env_path = os.path.join(os.getcwd(), ".env")

init_llm(prompts_path, env_path)

In [26]:
# Main agent builder that generates the agent for you
from worksheets import AgentBuilder, conversation_loop
import json
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="ERROR")

agent_builder = (
    AgentBuilder(
        name=botname,
        description=description,
        starting_prompt=starting_prompt
    )
    .with_gsheet_specification(gsheet_id_default)
)

agent = agent_builder.build(config)


In [23]:
for ws in agent.runtime.genie_worksheets:
  print(ws)

RequestHint(collectible_id: str, hint_tier: Enum['1', '2', '3'], confirm_details: bool)


# INTERACTION WITH AGENT

In [27]:
with agent:
  await conversation_loop(agent, debug=True)

Agent: Hello! I am CelesteCollectibleGenie. I can help you with hints and tips for collecting items in Celeste levels. How many strawberries have you collected so far? Check the pause menu in the top-left corner to see.

request_hint = RequestHint(collectible_id='strawberry_11', hint_tier='1', confirm_details=True)
User: strawberry_11
Agent: > Would you like to proceed with the provided game information for the collectible **strawberry_11**?



2025-12-01 15:55:54.334 | ERROR    | worksheets.core.runtime:execute:311 - Error: get_hint() missing 1 required positional argument: 'hint_tier'
2025-12-01 15:55:54.334 | ERROR    | worksheets.core.runtime:execute:312 - Code: __return = []
hint_message = get_hint(_obj.collectible_id.value, _obj.hint_tier.value)
__return.append(say(hint_message))


request_hint.confirm_details = True
User: strawberry_11
Agent: > Would you like to proceed with the provided game information for the collectible **strawberry_11**?
User: yes
Agent: > Since you confirmed, here’s your hint for the collectible **strawberry_11** at tier 1: 
>
> Look for areas where you can use your dash effectively to reach higher platforms. Pay attention to the environment for any hidden paths or tricky jumps that might lead you to the collectible. Good luck!

